In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\上市信息财务信息-成长能力指标.xlsx')

In [ ]:
data

In [ ]:
#将单位全部统一为以元为单位
def convert(cell):
    pattern1=(r'(-?\d+\.?\d+)亿')
    pattern2=(r'(-?\d+\.?\d+)万亿')
    pattern3=(r'(-?\d+\.?\d+)万')
    if re.findall(pattern1,cell)!=[]:
        cell=float(re.findall(pattern1, cell)[0])/10000
    elif re.findall(pattern2,cell)!=[]:
        cell=float(re.findall(pattern2, cell)[0])
    elif re.findall(pattern3,cell)!=[]:
        cell=float(re.findall(pattern3, cell)[0])/100000000
    else: cell=np.nan
    return cell

In [ ]:
data['营业总收入(元)']=data['营业总收入(元)'].map(convert)

In [ ]:
data['毛利润(元)']=data['毛利润(元)'].map(convert)

In [ ]:
data['归属净利润(元)']=data['归属净利润(元)'].map(convert)

In [ ]:
data['扣非净利润(元)']=data['扣非净利润(元)'].map(convert)

In [ ]:
data

In [ ]:
#将缺失值'--'变为nan
data=data.replace('--',np.nan)

In [ ]:
#查看出现较多的时间
data['日期'].value_counts()

In [ ]:
#补全时间，规整时间的格式
pattern='\d\d\d\d-\d\d-\d\d'
for i in range(len(data)):
    if re.findall(pattern,data.iloc[i,2])==[]:
        data.iloc[i,2]='20'+data.iloc[i,2]

In [ ]:
#去除企业编号与日期值同时相同的行，保留第一次出现的行
data.drop_duplicates(subset=['企业编号','日期'],keep='first',inplace=True)

In [ ]:
#再次查看出现较多的时间
data['日期'].value_counts()

In [ ]:
#将数量少于2800的日期全部丢弃
timelist=list(['2016-12-31','2017-09-30','2017-12-31','2018-06-30','2018-09-30','2017-06-30','2018-03-31','2017-03-31','2016-09-30'])
data=data[data['日期'].isin(timelist)]
data

In [ ]:
#转换数据格式
def get_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'[\d.]+',cell)
    if not _:
        return None
    return float(_[0])

In [ ]:
data.iloc[:,3:]=data.iloc[:,3:].applymap(get_number)

In [ ]:
data.sort_values(by=['企业编号','日期'],inplace=True)

In [ ]:
#查看排序结果是否达到目标效果
data.columns=pd.Index(['企业编号', '标题', '日期', '$营业总收入(元)', '$毛利润(元)', '$归属净利润(元)', '$扣非净利润(元)',
       '营业总收入同比增长(元)', '归属净利润同比增长(元)', '扣非净利润同比增长(元)', '营业总收入滚动环比增长(元)',
       '归属净利润滚动环比增长(元)', '扣非净利润滚动环比增长(元)'])

In [ ]:
data=data.set_index(['企业编号','日期'])

In [ ]:
data=data.unstack().reset_index().set_index('企业编号').T.drop('标题')
data=data.reindex(columns=list(range(1001,4001)))
data.to_pickle('上市信息财务信息-成长能力指标.pickle')

In [ ]:
data.index